In [1]:
import os
import numpy as np
from datetime import datetime
os.chdir("../")
from datasets import load_dataset, Dataset, DatasetDict
from src.data import DataProcessing, HF_Dataset
from src.utils import get_split_limit

/opt/mamba/lib/python3.10/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [2]:
dataset = load_dataset("monash_tsf", "traffic_hourly")
freq = "1H"

Found cached dataset monash_tsf (/home/onyxia/.cache/huggingface/datasets/monash_tsf/traffic_hourly/1.0.0/82998723d55d6edbc664cfaebe371004d849846ee5f61b57f1f974000d44c050)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
data_pipeline = DataProcessing(dataset)
# mtrain, mtest = data_pipeline.multi_variate_format(freq)

In [4]:
get_split_limit(10*[1., 2., 3.], np.array([0.7, 0.2, 0.1]))

array([21, 26, 30])

In [5]:
ds = HF_Dataset(
    [datetime(2023, 4, 12, 12, 0, 0), datetime(2023, 4, 12, 12, 0, 0)], 
    [10*[1, 2, 3], 10*[4, 5, 6]],
    np.array([0.7, 0.2, 0.1]),
    [10*[0, 0, 0], 10*[0, 0, 0]],
    [10*[None, None, None], 10*[None, None, None]]).getDatasetDict()

ds = HF_Dataset(
    [datetime(2023, 4, 12, 12, 0, 0)], 
    [10*[1, 2, 3]],
    np.array([0.7, 0.2, 0.1]),
    [10*[0, 0, 0]],
    [10*[None, None, None]]).getDatasetDict()

In [6]:
data_pipeline = DataProcessing(ds)
data_pipeline.multi_variate_format(freq)

([{'target': array([[1., 2., 3., 1., 2., 3., 1., 2., 3., 1., 2., 3., 1., 2., 3., 1.,
           2., 3., 1., 2., 3.]], dtype=float32),
   'feat_dynamic_real': array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
           nan, nan, nan, nan, nan, nan, nan, nan]], dtype=float32),
   'start': Period('2023-04-12 12:00', 'H'),
   'feat_static_cat': array([0], dtype=int32)}],
 [{'target': array([[1., 2., 3., 1., 2., 3., 1., 2., 3., 1., 2., 3., 1., 2., 3., 1.,
           2., 3., 1., 2., 3., 1., 2., 3., 1., 2., 3., 1., 2., 3.]],
         dtype=float32),
   'feat_dynamic_real': array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
           nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
           nan, nan, nan, nan]], dtype=float32),
   'start': Period('2023-04-12 12:00', 'H'),
   'feat_static_cat': array([0], dtype=int32)}])